In [33]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import psycopg2

## __ Work around to load bignumbers from json when using pandas __ ##
import pandas.io.json

# monkeypatch using standard python json module
import json
pd.io.json._json.loads = lambda s, *a, **kw: json.loads(s)
# monkeypatch using faster simplejson module
import simplejson
pd.io.json._json.loads = lambda s, *a, **kw: simplejson.loads(s)
# normalising (unnesting) at the same time (for nested jsons)
pd.io.json._json.loads = lambda s, *a, **kw: pandas.io.json.json_normalize(simplejson.loads(s))


# Summary

This is an analysis originated from a bounty from [DiamondDAO](https://docs.diamonddao.xyz/welcome/values) that aims to analyze the data coming form ETH Denver attendees and their pathway in crypto after the event.

**Main objective:** 
- To determine the profile of inviduals attending the event.
- To understand the impacts of the event in the ecosystem. 

# Data loading and cleaning

In [ ]:
# loading the poap data

poap_xdai = pd.read_json('datasets/xdai_token_data.json')
poap_xdai['chain'] = 'xdai'

poap_eth = pd.read_json('datasets/ethereum_token_data.json')
poap_eth['chain'] = 'ethereum'

poap = pd.concat([poap_eth, poap_xdai])

poap.info(null_counts=True)


In [ ]:
# loading DAO activity and membership data


In [ ]:
# loading token holder info data

# Analysis

Things to check:

- What is the proportion of ETH Denver POAP holders vs SPORK DAO holders.. the difference between these two populations
- What is the impact of the event in first time attendees? Is this introducing people to the ecosystem? what about continous participants
- What is the % of people that sticks to the event overtime


In [ ]:
# Check all ETH Denver POAP recipients
## What other events these people go? Any identifiable category or pattern here? 
## What is the rentetion rate for ETH Denver? 
## Does ETH Denver lead to other events in a strong way?

# Cross reference POAP holders and SPORK holders
## make analysis on how much of the the POAP issuers have token of the opposite - which parameters identify the most people?


# Isolate all possible ETH Denver participants and start looking at DAO cross reference like:
## how many DAOs can the person be identified at (DAOHaus)
## how many proposals or votes this people have? once you have proposals or votes this would add you as member of the DAO
## explore the concept of stronger membership and weak membership attached to how much people participate 
## explore level of activity
### how many votes
### how many different DAOs
### total number of interactions vs count of specific reactions: vote, proposals, etc
### what is the SPORK holders share in the votes in snapshot - what is their wight on these things? vs only POAP vs general public

